In [1]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['용어 설명']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess
text_dict = dict()
pattern = r'\((.*?)\)\/\(.*?\)'
#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        local_word_dict = set()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            dialogs = line['Dialogs']
            word_info = [[{'Word': word['Word'], 'WordDefine': word['WordDefine']} for word in dialog['WordInfo']]
                         for dialog in dialogs if dialog['WordInfo']]
            word_info = sum(word_info, [])
            word_info_dict = {}
            for info in word_info:
                word = info['Word']
                if word not in word_info_dict:
                    word_info_dict[word] = info['WordDefine'].strip().replace('\"', '')
                    
            text = '\n'.join([utt['SpeakerText'] for utt in dialogs])
            text = re.sub(pattern, r'\1', text)
            word_set = str(sorted(word_info_dict.keys()))
            if (word_set_text:=word_set) in local_word_dict:
                continue
            local_word_dict.add(word_set_text)
            if text[-20:] in text_dict and text_dict[text[-20:]] == word_set:
                continue
            text_dict[text[-20:]] = word_set
            
            #### data preprocess end 


            #### 용어 설명
            data = {'input': None, 'output': None}
            ## preprocess data from line
            data['input'] = text
            data['output'] = word_info_dict
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 용어 설명 end


            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 63/63 [00:01<00:00, 55.95it/s] 
